# Raster acquisition, processing and analysis with Databricks

The final notebook in this series will demonstrate how to:
- Use Mosaic's map algebra functions [↗︎](https://databrickslabs.github.io/mosaic/api/raster-functions.html) to compute the pixel-level Normalized Difference Vegetation Index (NDVI) and aggregate this to a single result per AoI.

## Install the libraries and prepare the environment

For this demo we will require a few spatial libraries that can be easily installed via pip install. We will be using gdal, rasterio, pystac and databricks-mosaic for data download and data manipulation. We will use planetary computer as the source of the raster data for the analysis.

In [0]:
import os

notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
project_path = os.path.dirname(notebook_path)
os.environ["PROJECTCWD"] = project_path

%pip install /Workspace$PROJECTCWD/databricks_mosaic-0.4.3-py3-none-any.whl
%pip install --quiet rasterio==1.3.5 gdal==3.4.1 pystac pystac_client planetary_computer tenacity rich osdatahub

In [0]:
%reload_ext autoreload
%autoreload 2

In [0]:
import library
import mosaic as mos
import os

from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql import Window

data_product = "OpenGreenspace"

current_user = spark.sql("select current_user() as user").first()["user"]
data_root = f"/tmp/{current_user}/{data_product}/data"
output_path = data_root.replace("/data", "/outputs")

dbutils.fs.mkdirs(data_root)
dbutils.fs.mkdirs(output_path)

os.environ["DATADIR"] = f"/dbfs{data_root}"
os.environ["OUTDIR"] = f"/dbfs{output_path}"

CATALOG = "marcell"
SCHEMA = "geospatial"

spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")

In [0]:
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "false")
mos.enable_mosaic(spark, dbutils)
mos.enable_gdal(spark, with_checkpoint_path=f"/dbfs{output_path}/checkpoint/{datetime.now().isoformat()}")

## Retrieve latest state from image processing workflow

In [0]:
clipped_image_table_ref = f"{CATALOG}.{SCHEMA}.clipped_rasters"
clipped_images = spark.table(clipped_image_table_ref)
clipped_images.display()

## Local map algebra functions

For calculating measures like Normalized Difference Vegetation Index (NDVI), we might need to apply some local map algebra function (i.e. transform and combine values from individual pixels in the same location across multiple raster bands).
- Mosaic has a built-in method for computing NDVI, `rst_ndvi()`[](https://databrickslabs.github.io/mosaic/api/raster-functions.html#rst_ndvi), but also allows for users to perform any local map algebra they like using the `rst_mapalgebra()` function.
- The return data type of these functions will match that of the input raster bands. This is a problem if our inputs are expressed as integers and our map algebra function requires division, since the output really should be a floating point number. To change the input bands to another data type, we can use the `rst_updatetype()` expression.

In [0]:
ndvi_table_ref = f"{CATALOG}.{SCHEMA}.ndvi"

(
  clipped_images
  .repartition(sc.defaultParallelism * 10)
  .withColumn("tile", mos.rst_updatetype("tile", F.lit("Float32")))
  .withColumn("ndvi", mos.rst_ndvi("tile", F.lit(4), F.lit(8)))
  .write
  .mode("overwrite")
  .saveAsTable(ndvi_table_ref)
  ) 

ndvi = spark.table(ndvi_table_ref)
ndvi.display()

In [0]:
ndvi.count()

## Examine results

We can examine the contents of checkpointed files using the `rasterio` and `matplotlib` python packages like this:

In [0]:
import matplotlib.pyplot as plt
import rasterio

def plot_file(file_path, bands):
  fig, ax = plt.subplots(1, figsize=(16, 16))

  with rasterio.open(file_path) as src:
    arr = src.read(bands, masked=True)
    rasterio.plot.show(arr, ax=ax, transform=src.transform, adjust=True)
    plt.show()
  
def plot_file_with_hist(file_path, band):
  fig, (axrgb, axhist) = plt.subplots(1, 2, figsize=(14,7))

  with rasterio.open(file_path) as src:
    arr = src.read(band, masked=True)
    rasterio.plot.show(arr, ax=axrgb, transform=src.transform, adjust=True)
    rasterio.plot.show_hist(arr, bins=50, histtype='stepfilled', lw=0.0, stacked=False, alpha=0.3, ax=axhist)
  plt.show()

In [0]:
def get_example(df):
  example_key = "1E275104-7485-3552-E063-AAEFA00A7A80"
  return df.where(f"id = '{example_key}'")

def get_path(df, colName):
  return df.transform(get_example).first()[colName]["raster"]

print(get_path(df=clipped_images, colName="tile"))
plot_file(get_path(df=clipped_images, colName="tile"), bands=[4, 3, 2])

In [0]:
plot_file(get_path(df=ndvi, colName="ndvi"), bands=1)

## Apply zonal map algebra functions

In contrast to the 'local' map algebra functions, their 'zonal' equivalents aggregate the pixel values of the entire raster tile.

The following functions are implemented:
- `rst_min()`
- `rst_max()`
- `rst_avg()` (computes the mean pixel value)
- `rst_median()`
- `rst_pixelcount()`

If the `tile` object passed to any of these functions is a multiband raster, an array of values will be returned.

In [0]:
(
  clipped_images
  .withColumn("max_per_band", F.expr("rst_max(tile)"))
  ).display()

The final part of this workflow is to take our computed NDVI tiles and turn them into a single value. In this case, we'll compute the mean value to try and express the relative healthiness of the vegetation of each green space.

We can then sort these values to identify the healthiest (or least healthy) golf courses in our great nation.

In [0]:
mean_ndvi = (
  ndvi
  .withColumn("ndvi_mean_test", F.expr("try_sql(rst_avg(ndvi))"))
  .where("ndvi_mean_test.status='OK'")
  .select(
    "id", "function", "distinctive_name_1", "geometry_4326",
    "ndvi.metadata.path",
    F.explode(F.col("ndvi_mean_test.result")).alias("ndvi_mean")
    )
).cache()

Let's look at the areas with the highest and lowest NDVI and see what we can conclude...

In [0]:
mean_ndvi.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{CATALOG}.{SCHEMA}.mean_ndvi")

In [0]:
mean_ndvi.display()

In [0]:
high_ndvi_id = (
  mean_ndvi
  .orderBy(F.desc("ndvi_mean"))
  .first()["id"]
  )

high_mean = mean_ndvi.where(f"id='{high_ndvi_id}'").first()
high_ndvi_row = ndvi.where(f"id='{high_ndvi_id}'").first()
high_ndvi_original_row = clipped_images.where(f"id='{high_ndvi_id}'").first()

print(f"ID: {high_mean['id']}")
print(f"Location: {high_mean['distinctive_name_1']}")
print(f"Mean NDVI: {high_mean['ndvi_mean']}")

plot_file(high_ndvi_original_row["tile"]["raster"], [4, 3, 2])
plot_file_with_hist(high_ndvi_original_row["tile"]["raster"], 4)
plot_file_with_hist(high_ndvi_original_row["tile"]["raster"], 8)
plot_file_with_hist(high_ndvi_row["tile"]["raster"], 1)

high_ndvi_row["geometry_4326"]

In [0]:
low_ndvi_id = (
  mean_ndvi
  .where(~(F.col("id")=="10687FF5-AFE1-64A4-E063-AAEFA00A3B27"))
  .orderBy("ndvi_mean")
  .first()["id"]
  )

low_mean = mean_ndvi.where(f"id='{low_ndvi_id}'").first()
low_ndvi_row = ndvi.where(f"id='{low_ndvi_id}'").first()
low_ndvi_original_row = clipped_images.where(f"id='{low_ndvi_id}'").first()

print(f"ID: {low_mean['id']}")
print(f"Location: {low_mean['distinctive_name_1']}")
print(f"Mean NDVI: {low_mean['ndvi_mean']}")

plot_file(low_ndvi_original_row["tile"]["raster"], [4, 3, 2])
plot_file_with_hist(low_ndvi_original_row["tile"]["raster"], 4)
plot_file_with_hist(low_ndvi_original_row["tile"]["raster"], 8)
plot_file_with_hist(low_ndvi_row["tile"]["raster"], 1)

low_ndvi_row["geometry_4326"]